In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_table('/projects/datatrans/krobasky/output/observation_fact.tsv', sep='\t',
                   header=0, index_col=False, nrows=10)
obs_columns = df.columns[:15]
df = pd.read_table('/projects/datatrans/krobasky/output/observation_fact.tsv',
                   header=0, index_col=False, nrows=100000, usecols=obs_columns)
col_dtypes = df.dtypes.to_dict()
col_dtypes['location_cd'] = np.dtype('O')
print col_dtypes

{'concept_cd': dtype('O'), 'location_cd': dtype('O'), 'provider_id': dtype('O'), 'end_date': dtype('O'), 'nval_num': dtype('float64'), 'instance_num': dtype('int64'), 'valueflag_cd': dtype('O'), 'units_cd': dtype('O'), 'patient_num': dtype('int64'), 'tval_char': dtype('O'), 'valtype_cd': dtype('O'), 'modifier_cd': dtype('O'), 'start_date': dtype('O'), 'quantity_num': dtype('float64'), 'encounter_num': dtype('int64')}


/home/mikeyu/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# df = pd.read_table('/projects/datatrans/krobasky/output/observation_fact.tsv', sep='\t',
#                    dtype=col_dtypes, header=0, usecols=obs_columns, index_col=False)

In [4]:
%%time

obs_df = pd.read_table('/projects/datatrans/krobasky/output/observation_fact.tsv', sep='\t', low_memory=True,
                   dtype=np.dtype('O'), header=0, usecols=obs_columns, index_col=False, skiprows)
print obs_df.shape

(54683532, 15)


In [7]:
obs_df.dropna(subset=['patient_num'], inplace=True)
print obs_df.shape

(54683531, 15)


In [12]:
print 'column', 'null', 'not_null'
for col in obs_df.columns:
    x = obs_df[col].isnull().sum()
    print col, x, df.shape[0] - x

encounter_num 0 54683531
patient_num 0 54683531
concept_cd 0 54683531
provider_id 1069120 53614411
start_date 0 54683531
modifier_cd 0 54683531
instance_num 0 54683531
valtype_cd 25268486 29415045
tval_char 25253285 29430246
nval_num 31473951 23209580
valueflag_cd 50201096 4482435
quantity_num 54683531 0
units_cd 33299626 21383905
end_date 41666759 13016772
location_cd 54683301 230


In [17]:
%%time

obs_df.to_pickle('/home/mikeyu/observation_fact.tsv.pkl', compression=None)

CPU times: user 3min 40s, sys: 13.1 s, total: 3min 53s
Wall time: 3min 53s


In [114]:
obs_df.reset_index(drop=True, inplace=True)

In [21]:
obs_df['start_date'] = pd.to_datetime(obs_df['start_date'])

In [96]:
obs_df['patient_num'] = obs_df['patient_num'].astype(np.int64)

In [23]:
obs_df.sort_values(['patient_num', 'start_date'], inplace=True)

In [139]:
tmp2 = obs_df['start_date']
obs_df['timedelta'].iloc[1:] = tmp2.iloc[1:].values - tmp2.iloc[:-1].values
obs_df.loc[0, 'timedelta'] = pd.Timedelta(0)

/home/mikeyu/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [84]:
(obs_df['start_date'] > pd.Timestamp('2017-10-27')).value_counts()

False    54675967
True         7564
Name: start_date, dtype: int64

In [168]:
# gap = pd.Timedelta(1, unit='D')
gap = pd.Timedelta(5, unit='D')
tmp = obs_df['patient_num'].values + np.cumsum((obs_df['timedelta'] <= gap).values)
obs_df['groupby_index'] = tmp
print 'Unique (patient, time-window) pairs', obs_df['groupby_index'].unique().size

Unique (patient, time-window) pairs 49503863


In [176]:
tmp = obs_df[['patient_num', 'timedelta']].groupby(by='patient_num')
tmp2 = tmp.aggregate(lambda x: np.unique(x['patient_num'].values + np.cumsum((x['timedelta'] <= gap).values)).size)

In [170]:
obs_df['timedelta'].unique().size

8769

In [75]:
tmp = obs_df.head(50).reset_index(drop=True)
gap = pd.Timedelta(1, unit='D')

for x, y in zip(tmp['patient_num'], tmp['timedelta'][:100] <= gap):
    print x, y

100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
100003190 True
